In [ ]:

import numpy as np
from scipy.integrate import solve_ivp
import time

# --- Định nghĩa lại các thành phần cần thiết ---
H0_matrix = H_0.to_matrix()
Dz_matrix = z_dipole_qubit.to_matrix()
dim = H0_matrix.shape[0]
U0_flat = np.eye(dim, dtype=complex).flatten() # U(0) = I, làm phẳng cho solver

# Hàm E-field
E0_val = 0.01
Gamma_val = 0.25
def E_field(t):
    if t < -200 or t > 200: return 0.0
    return (E0_val / np.pi) * Gamma_val / (Gamma_val**2 + t**2)

# Hàm H(t)
def H_t_matrix(t):
    return H0_matrix + E_field(t) * Dz_matrix

# --- Định nghĩa vế phải của PT vi phân cho Unitary: dU/dt = -i * H(t) * U ---
def unitary_rhs(t, U_flat):
    U = U_flat.reshape(dim, dim) # Chuyển vector về ma trận
    H = H_t_matrix(t)
    dU_dt = -1j * (H @ U)
    return dU_dt.flatten() # Chuyển về vector cho solver

# --- Thiết lập thời gian và chạy solver ---
# Dùng các điểm thời gian này để huấn luyện
times_for_training = np.linspace(0, 300, 50) 
t_span = [times_for_training[0], times_for_training[-1]]

print("Bắt đầu tính toán Target Unitaries bằng ODE Solver...")
start_time = time.time()
sol_U = solve_ivp(
    unitary_rhs,
    t_span,
    U0_flat,
    t_eval=times_for_training,
    method='RK45', # Phương pháp Runge-Kutta bậc 4(5)
    rtol=1e-8, # Dung sai cao để có kết quả chính xác
    atol=1e-8
)
end_time = time.time()
print(f"Hoàn thành trong {end_time - start_time:.2f} giây.")

# --- Xử lý kết quả ---
target_unitaries_list = []
if sol_U.success:
    # sol_U.y có dạng (dim*dim, N_points)
    unitaries_flat = sol_U.y.T # Chuyển vị thành (N_points, dim*dim)
    for u_flat in unitaries_flat:
        target_unitaries_list.append(u_flat.reshape(dim, dim))
    print(f"Đã tạo thành công {len(target_unitaries_list)} target unitaries.")
else:
    print("Lỗi: Bộ giải ODE không hội tụ khi tính toán Unitary.")
